In [18]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [19]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨.csv')
customer = pd.read_csv('해운대 입장객수2.csv')

In [20]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import Bidirectional
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
import inspect

# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수'] = customer['방문객수'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

In [21]:
# 데이터 합치기
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수"]]

In [22]:
# 결측치 평균, 0 채우기
total_data.loc[total_data['평균수온'] != total_data['평균수온'], '평균수온'] = total_data['평균수온'].mean()
total_data.loc[total_data['평균풍속'] != total_data['평균풍속'], '평균풍속'] = total_data['평균풍속'].mean()
total_data.loc[total_data['평균기압'] != total_data['평균기압'], '평균기압'] = total_data['평균기압'].mean()
total_data.loc[total_data['평균최대파고'] != total_data['평균최대파고'], '평균최대파고'] = total_data['평균최대파고'].mean()
total_data.loc[total_data['평균파주기'] != total_data['평균파주기'], '평균파주기'] = total_data['평균파주기'].mean()
total_data.loc[total_data['방문객수'] != total_data['방문객수'], '방문객수'] = total_data['방문객수'].mean()
# total_data = total_data.fillna(0)

## 방문객 수 예측

In [23]:
# input, output 지정
cos_x = total_data[['강수_관측값',"기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기","방문객수"]]
cos_y = total_data[['방문객수']]

In [24]:
# 방문객 수 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(cos_x,cos_y, test_size=0.2)

In [25]:
# 방문객 수 예측을 위한 scaler
cos_scaler =MinMaxScaler()
train_x = cos_scaler.fit_transform(train_x)
test_x = cos_scaler.transform(test_x)

In [26]:
train_x.shape,test_x.shape #((1172, 10), (293, 10))

((1172, 10), (293, 10))

In [27]:
train_x=train_x.reshape(1172,2,5)
test_x=test_x.reshape(293,2,5)

In [28]:
train_x.shape,test_x.shape

((1172, 2, 5), (293, 2, 5))

In [29]:
# 2. 모델구성 / 방문객
model=Sequential()
model.add(LSTM(units=256,input_shape=(2,5),return_sequences=True))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 2, 256)            268288    
                                                                 
 bidirectional_1 (Bidirectio  (None, 1024)             3149824   
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 128)               131200    
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                      

In [ ]:
# 3. 컴파일, 훈련 / 방문객
earlyStopping=EarlyStopping(monitor='val_loss',patience=500,mode='min',verbose=1,restore_best_weights=True) 
model.compile(loss='mse',optimizer='adam')
hist=model.fit(train_x,train_y,epochs=3000,batch_size=16,validation_split=0.1, callbacks=[earlyStopping])

Epoch 1/3000
66/66 [==============================] - 19s 136ms/step - loss: 28210241536.0000 - val_loss: 29535270912.0000
Epoch 2/3000
66/66 [==============================] - 5s 76ms/step - loss: 23497994240.0000 - val_loss: 19825606656.0000
Epoch 3/3000
66/66 [==============================] - 5s 76ms/step - loss: 14743996416.0000 - val_loss: 16887531520.0000
Epoch 4/3000
66/66 [==============================] - 5s 76ms/step - loss: 14222627840.0000 - val_loss: 16880852992.0000
Epoch 5/3000
66/66 [==============================] - 5s 78ms/step - loss: 14215943168.0000 - val_loss: 16886987776.0000
Epoch 6/3000
66/66 [==============================] - 5s 76ms/step - loss: 13953126400.0000 - val_loss: 16152543232.0000
Epoch 7/3000
66/66 [==============================] - 5s 76ms/step - loss: 13329133568.0000 - val_loss: 15230189568.0000
Epoch 8/3000
66/66 [==============================] - 5s 76ms/step - loss: 12418136064.0000 - val_loss: 12663090176.0000
Epoch 9/3000
66/66 [==========

In [16]:
# 4. 평가, 예측 / 방문객
loss=model.evaluate(test_x,test_y)
pred_y=model.predict(test_x)

print('loss: ',loss)
print('예상 입장객 수: ', pred_y[-1:])

10/10 [==============================] - 0s 20ms/step - loss: 2515360.0000
loss:  2515360.0
예상 입장객 수:  [[120265.625]]


loss:  10817543168.0
예상 입장객 수:  [[89993.17]] 
-random state 지정 / 에포 300 / 페이션트 50
loss:  11032627200.0
예상 입장객 수:  [[65531.715]]
-random state 빼봄 / 같은조건
loss:  10427985920.0
예상 입장객 수:  [[244709.86]]
-minmax로 바꿔보자
loss:  10741933056.0
예상 입장객 수:  [[61856.137]]
-data 바꿈!
loss:  1966186.0
예상 입장객 수:  [[6984.8716]]

loss:  2515360.0
예상 입장객 수:  [[120265.625]]

## 날씨_예측

In [7]:
# input data
weather_input = total_data[['강수_관측값',"기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = total_data[['강수_관측값','기온']]

In [8]:
# 강수_관측값 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input, weather_output, test_size=0.2, random_state=42)

In [9]:
# 강수_관측값 예측을 위한  minmax scaler
min_scaler = MinMaxScaler()
train_x_scaled = min_scaler.fit_transform(train_x)
test_x_scaled = min_scaler.transform(test_x)

In [10]:
train_x_scaled.shape,test_x_scaled.shape

((1172, 9), (293, 9))

In [11]:
train_x = train_x_scaled.astype(float)
test_x = test_x_scaled.astype(float)
train_y = train_y.astype(float)
test_y = test_y.astype(float)

In [12]:
train_x = train_x_scaled.reshape(1172, 3, 3)
test_x = test_x_scaled.reshape(293, 3, 3)

In [13]:
train_x.shape, test_x.shape

((1172, 3, 3), (293, 3, 3))

In [14]:
# 2.모델구성 / 날씨
model=Sequential()
model.add(LSTM(units=256,input_shape=(3,3),return_sequences=True))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256,activation='swish'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 256)            266240    
                                                                 
 bidirectional (Bidirectiona  (None, 1024)             3149824   
 l)                                                              
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                        

In [15]:
# 3. 컴파일 훈련 / 날씨
earlyStopping = EarlyStopping(monitor='val_loss', patience=500, mode='min', verbose=1,restore_best_weights=True)
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
hist = model.fit(train_x, train_y, epochs=3000, batch_size=1024, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 501/3000
1/1 - 1s - loss: 0.8757 - val_loss: 0.4787 - 1s/epoch - 1s/step
Epoch 502/3000
1/1 - 1s - loss: 0.5155 - val_loss: 0.4594 - 1s/epoch - 1s/step
Epoch 503/3000
1/1 - 1s - loss: 0.4872 - val_loss: 0.7311 - 1s/epoch - 1s/step
Epoch 504/3000
1/1 - 1s - loss: 0.7638 - val_loss: 0.6530 - 1s/epoch - 1s/step
Epoch 505/3000
1/1 - 1s - loss: 0.6675 - val_loss: 0.8449 - 1s/epoch - 1s/step
Epoch 506/3000
1/1 - 1s - loss: 0.8805 - val_loss: 0.4814 - 1s/epoch - 1s/step
Epoch 507/3000
1/1 - 1s - loss: 0.5017 - val_loss: 0.4638 - 1s/epoch - 1s/step
Epoch 508/3000
1/1 - 1s - loss: 0.4808 - val_loss: 0.6740 - 1s/epoch - 1s/step
Epoch 509/3000
1/1 - 1s - loss: 0.6674 - val_loss: 0.5012 - 1s/epoch - 1s/step
Epoch 510/3000
1/1 - 1s - loss: 0.5187 - val_loss: 0.3993 - 1s/epoch - 1s/step
Epoch 511/3000
1/1 - 1s - loss: 0.4294 - val_loss: 0.4237 - 1s/epoch - 1s/step
Epoch 512/3000
1/1 - 1s - loss: 0.4683 - val_loss: 0.5233 - 1s/epoch - 1s/step
Epoch 513/3000
1

In [17]:
# 4. 평가, 예측 / 날씨
loss=model.evaluate(test_x,test_y)
pred_y=model.predict(test_x)

print('loss: ',loss)
print('예상 강수량, 기온 : ', pred_y[-1:])

10/10 [==============================] - 0s 24ms/step - loss: 0.0315
loss:  0.03154443949460983
예상 강수량, 기온 :  [[ 0.08142194 25.326422  ]]


loss:  0.02682594209909439
예상 강수량, 기온 :  [[63.300068 22.007986]] 